### Importing all the libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

### Creating a Convolutional Neural Network Model

In [2]:
class CNN(nn.Module):
    
    def __init__(self, in_channels, num_classes):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1 = nn.Linear(16*7*7,num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        return x

### Checking the model on random data

In [3]:
model = CNN(1,10)
x = torch.randn(64,1,28,28)
print(model(x).shape)

torch.Size([64, 10])


In [4]:
arr = model(x)

In [5]:
arr

tensor([[ 0.0411,  0.0094, -0.0314, -0.1825,  0.0867,  0.5282,  0.0913,  0.3074,
          0.1280, -0.1461],
        [ 0.0981,  0.0708,  0.0584, -0.1803,  0.3383,  0.3673,  0.0811,  0.2631,
          0.1607, -0.0870],
        [-0.0047, -0.0255,  0.1623, -0.0859,  0.2009,  0.6980,  0.1100,  0.2595,
          0.2308, -0.2250],
        [ 0.1517, -0.0834,  0.0278, -0.2252,  0.0855,  0.6027, -0.0635,  0.2982,
          0.1572, -0.0416],
        [-0.0080, -0.0575,  0.1462, -0.2526,  0.2703,  0.4847,  0.1044,  0.1317,
          0.1023, -0.2084],
        [-0.0118,  0.2010,  0.1356, -0.0351,  0.0799,  0.6375,  0.1318,  0.3176,
          0.1485, -0.2061],
        [ 0.1211,  0.0342,  0.0326, -0.1950,  0.0959,  0.4840,  0.0810,  0.2414,
          0.0883,  0.0161],
        [ 0.0117, -0.0040,  0.0116, -0.2754, -0.0091,  0.6615,  0.1393,  0.2580,
          0.0986, -0.1880],
        [ 0.0221,  0.0274,  0.1462, -0.1058,  0.2850,  0.6409,  0.2712,  0.2371,
          0.1953, -0.0550],
        [-0.0730,  

In [6]:
torch.einsum("ij->i",arr)

tensor([0.8319, 1.1704, 1.3204, 0.9095, 0.7132, 1.3989, 0.9996, 0.7044, 1.6645,
        0.8009, 1.1476, 1.3468, 1.1150, 0.8487, 1.2312, 1.1181, 0.9168, 1.1009,
        1.3867, 0.9043, 0.9526, 0.8172, 1.0336, 1.0213, 1.1396, 0.5411, 0.9943,
        1.3095, 0.8135, 1.0127, 0.4492, 0.6911, 0.5611, 1.1731, 1.0910, 0.8701,
        1.0033, 1.3779, 1.4362, 1.1719, 1.4479, 1.0139, 0.8906, 1.5346, 0.7332,
        1.5635, 0.6002, 0.9558, 1.3564, 0.8010, 0.9696, 0.7997, 1.0456, 0.9524,
        1.0685, 0.9593, 0.8855, 0.5977, 1.2372, 1.0599, 1.2116, 1.1074, 0.9794,
        0.9930], grad_fn=<SumBackward1>)

### Using CUDA if it is available

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Model Parameters

In [8]:
in_channels = 1
num_classes = 10
learning_rate = 0.0001
batch_size = 64
num_epochs = 5

### Data Loading

In [9]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='data/',train=True, transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='data/',train=False, transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

### Intialize the Model

In [10]:
model = CNN(in_channels=in_channels,num_classes=num_classes).to(device)

### Loss and Optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train Network

In [ ]:
for epoch in range(num_epochs):
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get the data to CUDA if possible
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward
        scores = model(data)
        loss = criterion(scores,targets)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Descent
        optimizer.step()

### Checking the accuracy of the Model

In [ ]:
def check_accuracy(loader, model):
    
    if loader.dataset.train:
        print("Checking accuracy on training data")
        
    else :
        print("Checking accuracy on test data")
    
    num_correct = 0
    num_samples = 0
    
    #model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x)
            
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_samples += predictions.size(0)
            
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
        
    #model.train()

check_accuracy(train_loader,model)
check_accuracy(test_loader,model)

Checking accuracy on training data
